# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

mongoimport --db uk_food --collection establishments --file /Users/gurpreetsingh/Desktop/nosql-challenge/Resources/establishments.json --jsonArray

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from decimal import Decimal

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('650816e2ae2d887518162c88'),
 'FHRSID': 429982,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000043978',
 'BusinessName': 'The Barn',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Reach Court Farm',
 'AddressLine2': 'Reach Road',
 'AddressLine3': 'St Margarets At Cliffe',
 'AddressLine4': 'Kent',
 'PostCode': 'CT15 6AQ',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2017-09-21T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.371115', 'latitude': '51.147379'},
 'RightToReply': '',
 'Distance': 4587.77622109052,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0

In [7]:
# assign the collection to a variable
collection = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "146A Plumstead Rd",
    "AddressLine2": "Penang Flavours",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
collection.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
result = collection.find_one({"BusinessName": "Penang Flavours"})
if result:
    print("The new restaurant was successfully inserted.")
else:
    print("The new restaurant was not found in the collection.")

The new restaurant was successfully inserted.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
if result:
    print(result)
else:
    print("No matching document found.")

{'_id': ObjectId('650816e2ae2d887518162c88'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID

collection.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": "your_business_type_id"}})

In [14]:
# Confirm that the new restaurant was updated
result = collection.find_one({"BusinessName": "Penang Flavours"})
if result and result["BusinessTypeID"] == "your_business_type_id":
    print("The new restaurant was successfully updated.")
else:
    print("The new restaurant was not updated.")

The new restaurant was successfully updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
result = collection.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 994


In [17]:
# Check if any remaining documents include Dover
remaining_count = collection.count_documents({"LocalAuthorityName": "Dover"})
if remaining_count > 0:
    print("There are still documents with 'Dover' in the LocalAuthorityName field.")
else:
    print("There are no more documents with 'Dover' in the LocalAuthorityName field.")

There are no more documents with 'Dover' in the LocalAuthorityName field.


In [18]:
# Check that other documents remain with 'find_one'
other_document = collection.find_one({"BusinessName": {"$ne": "Penang Flavours"}})
if other_document:
    print("Other documents still exist in the collection.")
else:
    print("No other documents found.")

Other documents still exist in the collection.


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
from decimal import Decimal
# Convert Decimal objects to floats
latitude = float(Decimal("0"))
longitude = float(Decimal("0"))

# Update the latitude and longitude fields to decimal numbers
collection.update_many({}, {"$set": {"geocode.latitude": latitude, "geocode.longitude": longitude}})

In [20]:

# Check that the coordinates are now numbers
result = collection.find_one({"BusinessName": "Penang Flavours"})
if result:
    latitude = result["geocode"]["latitude"]
    longitude = result["geocode"]["longitude"]
    if isinstance(latitude, float) and isinstance(longitude, float):
        print("The coordinates are now numbers.")
    else:
        print("The coordinates are still not numbers.")
else:
    print("The document was not found.")

The coordinates are now numbers.


In [21]:
# Use update_many to convert RatingValue to integer numbers
update_operation = [
    {
        "$set": {
            "RatingValue": {
                "$cond": {
                    "if": {"$isNumber": "$RatingValue"},
                    "then": {"$toInt": "$RatingValue"},
                    "else": None
                }
            }
        }
    }
]

# Update the documents
update_result = collection.update_many({}, update_operation)

# Print the number of documents updated
print(f"Number of documents updated for RatingValue: {update_result.modified_count}")

Number of documents updated for RatingValue: 38786
